In [1]:
''' In this doc, we shall properly implement Mikes method. The previous version 
was too elementary and didnt have tractable gradients. '''

import numpy as np
import pandas as pd


In [12]:
#We shall now defined the number of input variables as well as the number of observations:

m = 4 #No. variables
n = 400 #No. observations


#NOTE, this will be the number of observations for both the "grid" on which we instantiate the MV Gaussian,
#as well as the draw from this prior.

#Now we define the variance and lengthscales:

m_var = 1
m_len = range(1,m) #We shall have different size lengthscales for the different variables to predefine importance (ARD)

In [13]:
#Now create the grid of data:
X = []
for i in range(m):
    X.append(np.random.uniform(1,5,n))

X = np.array(X)
X = np.transpose(X)


In [14]:
np.shape(X)

(400, 4)

In [ ]:
#We shall extract a mean and covariance from this data:

mean = np.mean(X,axis=0)
covariance = np.cov(np.transpose(X))


In [ ]:
#We can also define a kernel and GP:

np.random.multivariate_normal(mean, covariance)

In [15]:
#I think what I've done so far isn't quite correct, I'm looking for a mean function and mean covariance

L = np.identity(m)
L[1,1] = 2
L[2,2] = 5
L[3,3] = 10

def compute_cov(x,y,var,l_matrix):  #x and y are each full observations (vectors)
    d = 0
    m = len(x)
    diff = x-y
    L = np.linalg.inv(l_matrix)
    exponent = -0.5*(np.matmul(np.transpose(diff),np.matmul(L,diff)))
    val = var*np.exp(exponent)
    return val
#This function takes in two observations and outputs a scalar



K = np.empty([n,n])

#Now lets start filling this matrix:



for i in range(n):
    for j in range(n):
        K[i,j] = compute_cov(X[i,:],X[j,:],m_var,L)

K



array([[1.00000000e+00, 2.07004911e-02, 9.52250674e-02, ...,
        3.52005865e-02, 1.13225676e-01, 3.70154081e-02],
       [2.07004911e-02, 1.00000000e+00, 2.41886288e-01, ...,
        2.29218683e-02, 8.83909578e-02, 1.31115370e-01],
       [9.52250674e-02, 2.41886288e-01, 1.00000000e+00, ...,
        7.37348798e-03, 3.09263479e-02, 7.97607432e-01],
       ...,
       [3.52005865e-02, 2.29218683e-02, 7.37348798e-03, ...,
        1.00000000e+00, 7.07069232e-01, 7.56237370e-04],
       [1.13225676e-01, 8.83909578e-02, 3.09263479e-02, ...,
        7.07069232e-01, 1.00000000e+00, 5.01393698e-03],
       [3.70154081e-02, 1.31115370e-01, 7.97607432e-01, ...,
        7.56237370e-04, 5.01393698e-03, 1.00000000e+00]])

In [16]:
#We can now define our prior GP with 0 mean function and covariance function K
#We can draw a sample from this:

mean = np.zeros((n,1))
mean = mean.reshape(n)

sample = np.random.multivariate_normal(mean, K)
sample

array([-3.21963629e-01,  3.68809718e-01, -1.95701111e+00, -2.13141393e+00,
        2.63182037e+00, -3.74558001e-01, -9.48590974e-01,  7.47534620e-01,
       -2.60655997e+00,  1.62217404e+00,  3.10918447e-01,  1.22344969e+00,
        6.80552919e-02,  5.66585246e-01,  8.57030306e-01, -2.34870940e-01,
        5.61658600e-01, -2.34049086e+00, -2.23950552e+00,  7.14828766e-01,
       -1.20140810e+00, -1.08095345e+00,  2.33320495e+00,  1.70693194e+00,
       -8.59463012e-01,  1.18502350e+00,  6.84908332e-01, -1.74153948e-01,
       -2.75097107e+00,  3.05824566e+00,  4.16198911e-01, -8.14992822e-01,
        1.65501656e-01, -4.40039920e-01, -1.80119362e+00, -1.46601024e+00,
        2.15641195e+00,  9.32640199e-01,  2.98971919e+00,  8.74311272e-01,
       -6.31199760e-01, -8.30508120e-03,  3.52698850e-01,  2.12637592e+00,
       -7.40012074e-02,  2.02829256e+00, -2.02409557e+00, -2.29391811e+00,
        4.26430758e-01, -1.10896194e+00,  7.69463111e-01, -1.41760353e+00,
        1.43301744e+00,  

In [17]:
#We can now squash this through a sigmoid function:

import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))
output=[]

for i in sample:
    output.append(sigmoid(i))

output=np.asarray(output)
output = np.round(output)

In [18]:
output

array([0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
       0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0.,
       0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1.,
       0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
       1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0.,
       1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1.,
       0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
       1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0.,
       1., 1., 1., 0., 1.

In [19]:
data = pd.DataFrame(X)
data['Output'] = output
data

,0,1,2,3,Output
0,3.835124,1.211726,1.624049,1.004157,0.0
1,2.071587,4.027580,2.457672,3.331458,1.0
2,2.426488,2.066718,4.414327,3.825741,0.0
3,1.352256,2.995869,4.370891,3.066830,0.0
4,3.517934,4.158626,3.029461,2.430427,1.0
...,...,...,...,...,...
395,2.603168,3.698404,2.274507,2.363148,1.0
396,2.270919,2.154934,1.775154,1.941480,0.0
397,4.787810,4.475554,1.872012,3.118846,1.0
398,4.182972,4.107640,1.916942,1.508439,1.0


In [20]:
data.to_csv("generated_data2.csv",index=False)

In [21]:
data.to_csv("C:/Users/jacob/Documents/4YP data/DataSynthesis/4x400/data.csv",index=False)